In [ ]:
import myfitnesspal
import datetime
from datetime import date
from datetime import timedelta
import numpy as np
import math 
USERNAME ='your_username'
PASS = 'password'

CLIENT = myfitnesspal.Client(USERNAME, password=PASS)

WINDOW = {'week1_ago': ((date.today()) - timedelta(days=7), date.today()- timedelta(days=1)),
          'week2_ago':((date.today()) - timedelta(days=14), date.today()- timedelta(days=8)),
          'week3_ago':((date.today()) - timedelta(days=21), date.today()- timedelta(days=15))}

def get_food():
    food_list = []
    for day in range((WINDOW['week1_ago'][1]- WINDOW['week2_ago'][0]).days+1):
        food = CLIENT.get_date(WINDOW['week2_ago'][0] + timedelta(days=day)).totals
        food_list.append(food)
    return food_list

def summarize_kcal():
    total_kcal = 0
    food_list = get_food()
    for entry in food_list:
        total_kcal += entry['calories']
    return total_kcal
    
def summarize_weight():
    weekly_averages = {}
    for week in WINDOW.keys():
        week_array = []
        weight = CLIENT.get_measurements('Weight', WINDOW[week][0], WINDOW[week][1])
        for day in range((WINDOW[week][1]-WINDOW[week][0]).days+1):
            week_array.append(weight[WINDOW[week][0]+timedelta(days=day)])
        weekly_averages[week] = np.mean(week_array)
    return weekly_averages

weekly_averages = summarize_weight()
weight_change = abs(weekly_averages['week1_ago'] - weekly_averages['week3_ago']) 
in_kcal = weight_change * 7700
kcal_2weeks = summarize_kcal()

if weekly_averages['week1_ago'] > weekly_averages['week3_ago']:
    final_tdee = (kcal_2weeks - in_kcal)/14
else:
    final_tdee = (kcal_2weeks + in_kcal)/14
    
print('Maintenance calories (daily) are:', final_tdee)

In [2]:
# Calories/ TDEE
#Take your average weekly weight 3 weeks ago and subtract your average weekly weight now. 
#Multiply this weight change by 3500 to get the 2 weekly surplus (if you gained weight) or deficit (if you lost weight). 
#For example 3.1lbs loss x 3500 = 10,850 cals deficit.  ( kg * 2.205)
#Add up your calorie intake from week 2 and week 3 and add your deficit or deduct your surplus. Whatever the total, divide it by 14 (days in 2 weeks) and you have your maintenance calories. 
tdee = final_tdee
weight = CLIENT.get_measurements('Weight', date.today())[date.today()]
deficit = 500
kcal_input = tdee-deficit
calories_from_protein = (4*weight)
calories_from_fat = kcal_input - calories_from_protein - 25
print('For Keto, with a defecit of 500kcal: \nTotal calorie intake =', round(kcal_input), 'of which:\n', round(calories_from_fat),'kcal are from fat, which is roughly',round(calories_from_fat/8),
      'grams \n',round(calories_from_protein),'kcal of protein, which is roughly', weight*1,'grams. \nTry to stay under 20g of carbs.')



For Keto, with a defecit of 500kcal: 
Total calorie intake = 978 of which:
 757 kcal are from fat, which is roughly 95 grams 
 196 kcal of protein, which is roughly 48.9 grams. 
Try to stay under 20g of carbs.
